<h1 style = "text-align: center;">Project - Handling Missing Data </h1>

### Created by Brian Butzen and Nicholas Thomson

Install the required libraries if they are not already installed

In [1]:
!pip install pandas

In [2]:
!pip install scikit-learn

Import the required Python libraries.

In [3]:
from tkinter import *
import math
import pandas as pd
from sklearn.linear_model import LinearRegression

## Substitution Methods

In [4]:
#Substitution with mode for categorical
def ReplaceCategoricalMode(DataFrame: pd.DataFrame) -> pd.DataFrame:
    for x in DataFrame: #Goes through each column in dataframe
        if(DataFrame[x].dtype == object): #Check to see if the datatype of the column is object
            mode = DataFrame[x].mode() #Find the mode of the column
            DataFrame[x] = DataFrame[x].fillna(mode[0]) #Set the NaN values in the column to the mode
    return DataFrame

#Substitution with mean for numerical
def ReplaceNumericalMean(DataFrame: pd.DataFrame) -> pd.DataFrame:
    for x in DataFrame: #Goes through each column in dataframe
        if(DataFrame[x].dtype == 'float64' or DataFrame[x].dtype == 'int64'): #Check to see if the datatype of the column is a number
            mean = DataFrame[x].mean() #Find the mean of the column
            DataFrame[x] = DataFrame[x].fillna(mean) #Set the NaN values in the column to the mean
    return DataFrame

#Substitution with median for numerical
def ReplaceNumericalMedian(DataFrame: pd.DataFrame) -> pd.DataFrame:
    for x in DataFrame: #Goes through each column in dataframe
        if(DataFrame[x].dtype == 'float64' or DataFrame[x].dtype == 'int64'): #Check to see if the datatype of the column is a number
            median = DataFrame[x].median() #Find the median of the column
            DataFrame[x] = DataFrame[x].fillna(median) #Set the NaN values in the column to the median
    return DataFrame

#Makes a CSV
def MakeCSV(DataFrame: pd.DataFrame, s):
    DataFrame.to_csv(s +'Out.csv', index=False)

#Substitution using predictive modeling
#Returns dataframe with predicted values of missing numerical columns
#Uses linear regression model to predict values
def PredictiveModeling(df: pd.DataFrame) -> pd.DataFrame:
    old_data = df #Keep old dataframe for modification
    df = pd.get_dummies(df) #Convert categorical to dummy variables
    for x in df:
        if df[x].dtype == 'float64' or df[x].dtype == 'int64': #Makes sure column is numerical
            if df[x].isnull().sum() == 0: #Checks if there is any NaN/missing values
                continue
            #x is the name of the column
            test_data = df[df[x].isnull()] #Testing data is every row that has nan value in the column
            dropped_na = df.dropna() #drop the nan values for linear regression model

            #Get columns to test
            x_train = dropped_na.drop(x, axis=1)
            y_train = dropped_na[x]

            #Fit linear regression model
            lr = LinearRegression()
            lr.fit(x_train, y_train)

            #Predict missing values using model
            x_test = test_data.drop(x, axis=1)
            y_pred = lr.predict(x_test)

            #Replace old data with new data
            newSeries = [] 
            count = 0
            #Index through old data, replace nan values with predicted values and keeping the old ones
            for y in old_data.index:
                if math.isnan(old_data.iloc[y][x]):
                    newSeries.append(y_pred[count])
                    count += 1
                else:
                    newSeries.append(old_data.iloc[y][x])
            old_data[x] = newSeries

    return old_data




## Missing Values Display

In [5]:
def missing(dataframe):
    #Open the file for writing, and create the file if it is not present.
    #All data will be overidden each time
    missing_file = open('Missing.txt', 'w')
    missing_file.write("The number of missing values in each column\n")
    #Write the missing values in each column using the isnull pandas method. Axis = 0 reports the columns
    missing_file.write(dataframe.isnull().sum(axis = 0).to_string())
    missing_file.write("\n\nThe data types of the columns are:\n")
    #Write the data types of each column using dtypes in pandas 
    missing_file.write(dataframe.dtypes.to_string())
    missing_file.write("\n\nThe number of missing values in each row is\n")
    #Write the missing values in each row using the isnull pandas method. Axis = 1 reports the rows
    missing_file.write(dataframe.isnull().sum(axis = 1).to_string())
    #Displaying the total number of missing values using isnull. The two sums totals the columns and the rows
    missing_file.write("\n\nThe total number of missing values is " + str(dataframe.isnull().sum().sum()))
    #Displaying the statistical values without the NA values present
    missing_file.write("\n\nStatistical values of the present data")
    missing_file.write("\nAge -            Mean: " + str(dataframe['Age'].dropna().mean()))
    missing_file.write("\n                 Median: " + str(dataframe['Age'].dropna().median()))
    missing_file.write("\nProduct Rating - Mean: " + str(dataframe['ProductRating'].dropna().mean()))
    missing_file.write("\n                 Median: " + str(dataframe['ProductRating'].dropna().median()))
    missing_file.write("\nGender -         Mode: " + str(dataframe['Gender'].dropna().mode()[0]))
    missing_file.write("\nEducation -      Mode: " + str(dataframe['Education'].dropna().mode()[0]))
    missing_file.write("\nOccupation -     Mode: " + str(dataframe['Occupation'].dropna().mode()[0]))
    missing_file.write("\nMarital Status - Mode: " + str(dataframe['MaritalStatus'].dropna().mode()[0]))
    missing_file.close()

## Create Reports

In [6]:
#callback function to call the create_report function when the button is presses
def run():
    create_report(master, v.get())

#calls the appropriate function depending on which radio button is pressed.
#The label is updated in the GUI displaying which CSV was created.
#
def create_report(master, v):
    dataframe = pd.read_csv("input.data")
    match v:
        case 1:
            missing(dataframe)
            label.config(text = "The report Missing.txt has been created.")
        case 2:
            print("Original data:")
            print(dataframe.to_string())
            df = ReplaceNumericalMean(dataframe)
            label.config(text = "The CSV MeanOut has been created.")
            print("Missing data replaced with Mean")
            print(df.to_string())
            MakeCSV(df, "Mean")
        case 3: 
            print("Original Data")
            print(dataframe.to_string())
            df = ReplaceNumericalMedian(dataframe)
            label.config(text = "The CSV MedianOut has been created.")
            print("Missing data replaced with Median:")
            print(df.to_string())
            MakeCSV(df, "Median")
        case 4:
            print("Original Data")
            print(dataframe.to_string())
            df = ReplaceCategoricalMode(dataframe)
            label.config(text = "The CSV ModeOut has been created.")
            print("Missing data replaced with Mode:")
            print(df.to_string())
            MakeCSV(df, "Mode")
        case 5:
            print("Original Data")
            print(dataframe.to_string())
            df = PredictiveModeling(dataframe)
            label.config(text = "The CSV RegressionOut has been created.")
            print("Missing data replaced with Regression")
            print(df.to_string())
            MakeCSV(df, "Regression")

## GUI Implementation

In [7]:
master = Tk()
master.title("Data Management Tool")
master.minsize(400, 300)
middle_frame = Frame(master)
bottom_frame = Frame(master)
v = IntVar(master, 1)
values = {"Missing Values" : 1,
          "Substitute With Mean": 2,
          "Substitute With Median": 3,
          "Substitute With Mode": 4,
          "Substitute With Regression": 5}

#For each loop to create the formatted radio buttons
for (text, value) in values.items():
    Radiobutton(master, text=text, variable=v, value=value, indicator = 0, background = "light blue").pack(fill = X, ipady = 5)

#Buttons for creation of the report and for closing the window
s_button = Button(bottom_frame, text="Generate Report", command = run)
cancel_button = Button(bottom_frame, text="Cancel", command=master.destroy)

#Label to display that the report was created
label = Label(master, text='', justify="center")
#pack all of the elements into the window
label.pack(side = TOP, expand=True)
middle_frame.pack(side= BOTTOM)
s_button.pack(side = LEFT)
cancel_button.pack(side = LEFT)
bottom_frame.pack(side = BOTTOM)

mainloop()


Original Data
    CustomerID  Gender   Age    Education          Occupation MaritalStatus  ProductRating
0         1001    Male  35.0   Bachelor's   Digital Marketing       Married            4.5
1         1002  Female  28.0   Bachelor's   Software Engineer        Single            3.8
2         1003    Male  45.0     Master's             Finance       Married            4.2
3         1004  Female  32.0  High School               Sales        Single            NaN
4         1005  Female   NaN     Master's             Teacher       Married            4.0
5         1006    Male  40.0    Doctorate          Operations       Married            4.7
6         1007    Male  38.0   Bachelor's           Marketing        Single            4.3
7         1008  Female  35.0     Master's                 NaN        Single            4.1
8         1009    Male  33.0  High School        Construction        Single            NaN
9         1010  Female   NaN   Bachelor's        Data Analyst       Married 